In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
import sklearn
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled = StandardScaler().fit_transform(df_market_data)
scaled

In [ ]:
# Create a DataFrame with the scaled data
df_scaled = pd.DataFrame(scaled, columns=df_market_data.columns)

# Copy the crypto names from the original DataFrame
df_scaled["coin_id"] = df_market_data.index

# Set the coin_id column as index
df_scaled = df_scaled.set_index("coin_id")

# Display the scaled DataFrame
df_scaled


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values from 1 to 11
ks = list(range(1, 12))
ks

In [ ]:
# Create an empty list to store the inertia values
inertiae = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for k in ks:
    k_model = KMeans(n_clusters=k, random_state=42)
    k_model.fit(df_scaled)
    inertiae.append(k_model.inertia_)

inertiae

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
inertia_dict = {ks[i]: inertiae[i] for i in range(len(ks))}

# Create a DataFrame with the data to plot the Elbow curve
df_inertia = pd.DataFrame.from_dict(inertia_dict, orient="Index", columns=["Inertia"])
df_inertia.index.name = "K Value"
df_inertia

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
scaled_elbow_plot = df_inertia.hvplot.line()
scaled_elbow_plot

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 4

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=42)

In [ ]:
# Fit the K-Means model using the scaled DataFrame
k_model.fit(df_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
clusters = k_model.predict(df_scaled)

# Print the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the scaled DataFrame
df_unreduced_final = df_scaled.copy()

In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_unreduced_final["Cluster"] = clusters

# Display the copy of the scaled DataFrame
df_unreduced_final

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
scaled_cluster_plot = df_unreduced_final.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols="coin_id"
)
scaled_cluster_plot

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
reduced_pca = pca.fit_transform(df_scaled)

# View the scaled PCA data
reduced_pca

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
print(pca.explained_variance_ratio_, pca.explained_variance_ratio_.sum())

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

In [ ]:
# Create a new DataFrame with the PCA data.
df_reduced = pd.DataFrame(reduced_pca, columns=["PC1", "PC2", "PC3"])

# Copy the crypto names from the original scaled DataFrame
df_reduced["coin_id"] = df_market_data.index

# Set the coin_id column as index
df_reduced = df_reduced.set_index("coin_id")

# Display the scaled PCA DataFrame
df_reduced

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
ks = list(range(1, 12))
ks

In [ ]:
# Create an empty list to store the inertia values
inertiae = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for k in ks:
    k_model = KMeans(n_clusters=k, random_state=42)
    k_model.fit(df_reduced)
    inertiae.append(k_model.inertia_)

inertiae

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
inertia_dict = {ks[i]: inertiae[i] for i in range(len(ks))}

# Create a DataFrame with the data to plot the Elbow curve
df_inertia_pca = pd.DataFrame.from_dict(inertia_dict, orient="Index", columns=["Inertia"])
df_inertia_pca.index.name = "K Value"
df_inertia_pca

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
pca_elbow_plot = df_inertia_pca.hvplot.line()
pca_elbow_plot

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialize the K-Means model using the best value for k
k_model = KMeans(n_clusters=4, random_state=42)

In [ ]:
# Fit the K-Means model using the PCA data
k_model.fit(df_reduced)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
clusters = k_model.predict(df_reduced)

# Print the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the scaled PCA DataFrame
df_reduced_final = df_reduced.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_reduced_final["Cluster"] = clusters

# Display the copy of the scaled PCA DataFrame
df_reduced_final

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Color the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_reduced_final.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols="coin_id"
)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
# Note from student: the Challenge instructions say we should use the Plus (+) operator to compose these graphs,
# but that puts them side-by-side. It's much easier to compare the two graphs in the same plot:
scaled_elbow_plot * pca_elbow_plot

In [ ]:
# Composite plot to contrast the clusters

# The previous cluster plots used different axes so it doesn't make much sense to put them together.
# So add the original columns back in to the PCA-reduced dataframe, then use them to graph the results.
combo_df = pd.concat([df_reduced_final, df_scaled], axis=1)
pca_cluster_plot = combo_df.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="Cluster",
    hover_cols="coin_id"
)

scaled_cluster_plot + pca_cluster_plot

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Visually there is no apparent difference, but this isn't a very good way to check. I confirmed this conclusion in the next cell.

In [ ]:
# Rigorously check for any differences in clustering between the two results
any(df_unreduced_final["Cluster"] != df_reduced_final["Cluster"])